For this reason, contigs correspond to strings spelled by maximal non-branching paths in the de Bruijn graph

In [1]:
from collections import defaultdict
import numpy as np

In [58]:
patterns = [s.strip() for s in list(open("rosalind_ba3k.txt", "r"))]
#print(len(patterns) - np.unique(patterns).size)
k = len(patterns[0])
#print(patterns)

graph = defaultdict(list)
indeg, outdeg = defaultdict(int), defaultdict(int)

nodes = []
for kmer in patterns:
    pref, suff = kmer[:-1], kmer[1:]
    outdeg[pref] += 1
    indeg[suff] += 1
    nodes.extend([pref, suff])
    graph[pref].append(suff)
is_ok = {v : indeg[v] == 1 and outdeg[v] == 1 for v in nodes}

#print(nodes)

contigs = set()
visited = defaultdict(int)
for u in nodes:
    #print(u)
    if not is_ok[u]:
        visited[u] = True
        if outdeg[u] > 0:
            for w in graph[u]:
                path = [u, w[-1]]
                #print(w[-1])
                v = w
                while is_ok[v]:
                    visited[v] = True
                    v = graph[v][0]
                    path.append(v[-1])
                contigs.add(''.join(path))
#print(contigs)
for u in nodes:
    if visited[u] > 0:
        continue
    assert indeg[u] == 1 and outdeg[u] == 1
    visited[u] = True
    cycle = [u]
    v = u
    while True:
        v = graph[v][0]
        assert indeg[v] == 1 and outdeg[v] == 1, print(u, v)
        visited[v] = True
        if v == u:
            break
        cycle.append(v[-1])
    contigs.add(''.join(cycle))

answer = '\n'.join(sorted(contigs))
print(answer)

AAAAATAAGCTAGGAGGG
AAAAGAACATTTCCTATGGCGGTTCAGCCCGC
AAAATAAGCTAGGAGGGA
AAATAAGCTAGGAGGGAAATGGAAGTTTAGC
AAATGGAAGTTTAGCTCT
AACCCCATGCTAGGATCGGCGTCCCTCCAGAAA
AAGCAAGATTAGAAGTCATTAGATTGCCGTG
AAGGGTAGTCGCGATAGTCTTAGTATGGTG
AAGTCCTTAATACATCGT
AATCCGCCGACGAAGGCC
AATCGTCTAAGTATCTAAGTCCTTAATACATC
AATCTCTTTCAACCGCGC
AATGGAAGTTTAGCTCTGGCTGGCAGTGTAT
ACACCGTTTGAGCGGATC
ACCGTCTTCAGGGCTACTCACTACTGCTTGG
ACCGTTTGAGCGGATCCC
ACTACGCTTGGGAGTTTC
ACTACTAAATGGTGGAAC
ACTCACTACTGCTTGGTG
ACTCCACCGGAAATGCTGAATCGGAGCGTAAGTATA
ACTGACCAGCTTCTCGCT
ACTTATCACAAGGTAAGGTTTCCGATAGTCGTGA
ACTTCGTAAGATGAATGT
AGACCGTCTTCAGGGCTA
AGCCTAGGGGTTGACCTG
AGTACATCGTCAGCAAACGTAATACCTTGACATA
AGTAGGGGGTCTTATACT
AGTCCTTAATACATCGTCCCCGCTGCCGTCTG
AGTCTTAGTATGGTGCAG
AGTGGATACACGGTACCG
AGTTTCTCAGAACCCTCA
ATAAGTAAAGTGTCCTCTGGTCATCAACACAAG
ATAATCTCTTTCAACCGC
ATACTACGCTTGGGAGTT
ATAGTCTTAGTATGGTGC
ATAGTTTCTCAGAACCCT
ATCAAATTCCCAGGTATAGTTTCTCAGAACC
ATCAGTCGACATTCTGCGAGAATCAAGTACGTGTGG
ATCCGCCGACGAAGGCCT
ATCGGAGCGTAAGTATAC
ATCTCGTCGGGGATCGGA
ATCT

In [75]:
def build_graph(Adj, patterns, in_degree, out_degree):
    vertices = []
    for kmer in patterns:
        prefix, suffix = kmer[:-1], kmer[1:]
        Adj[prefix].append(suffix)

        out_degree[prefix] += 1
        in_degree[suffix] += 1

        vertices.extend([prefix, suffix])

    non_branching = {v: in_degree[v] == 1 and out_degree[v] == 1 for v in vertices}

    return vertices, non_branching

In [76]:
def check_cycles(contigs, visited, vertices, in_degree, out_degree):

    for u in vertices:
        if visited[u] > 0:
            continue
            
        assert in_degree[u] == 1 and out_degree[u] == 1
        visited[u] = True
        cycle = [u]
        v = u
        while True:
            v = Adj[v][0]

            # every vertex in the cycle has to be balanced
            assert in_degree[v] == 1 and out_degree[v] == 1, print(u, v)
            visited[v] = True
            if v == u:
                break
            cycle.append(v[-1])
        contigs.add(''.join(cycle))

In [77]:
def find_contigs(Adj, non_branching, vertices, in_degree, out_degree):
    contigs = set()
    visited = defaultdict(int)
    for u in vertices:
        # if in_degree[u] != out_degree[u], then
        # u is the beginning/end of a contig
        if not non_branching[u]:
            visited[u] = True

            # we check for a non-branching path
            # if u is the beginning
            if out_degree[u] > 0:
                for w in Adj[u]:

                    # build non-branching path
                    path = [u, w[-1]]
                    v = w

                    # Along the path with DFS
                    while non_branching[v]:
                        visited[v] = True
                        v = Adj[v][0]
                        path.append(v[-1])

                    contigs.add(''.join(path))

    check_cycles(contigs, visited, vertices, in_degree, out_degree)

    return contigs

In [78]:
def contigs_generation(patterns):
    
    Adj = defaultdict(list)
    in_degree = defaultdict(int)
    out_degree = defaultdict(int)
    
    vertices, non_branching = build_graph(Adj, patterns, in_degree, out_degree)
    
    return find_contigs(Adj, non_branching, vertices, in_degree, out_degree)
    

In [79]:
def main():
    
    file = open('rosalind_ba3k.txt', 'r')
    
    patterns = []
    for s in file:
        patterns.append(s.strip())
    
    
    print(*contigs_generation(patterns))

    file.close()

In [80]:
if __name__ == '__main__':
    main()

CATTCGGGCCAGGTAGTT AATCCGCCGACGAAGGCC TGTATCGCCTCTTCTACATCAGTCGACATTCTG CCAAGCAAGATTAGAAGT ATAGTCTTAGTATGGTGC GCTCAGCGCCATATGCCCAAGCAAGATTAGAA GAATCGTCTAAGTATCTA ATGGCGGTTCAGCCCGCC ATCTCTTTCAACCGCGCA ACTGACCAGCTTCTCGCT AGTCCTTAATACATCGTCCCCGCTGCCGTCTG TGGCTGGCAGTGTATTTA CGTGGAGAAGATAAGCTCCATGTATCCTTTG CATTCATAAATTAAACGGGCGTGGAGAAGATA CTTGCCAGATTATATTAT TGATTCCAGCTAGGAGGCAGTACATCGTCAGCAA TTCCCCATTGCGCATAATCTCTTTCAACCG CATCAGTCGACATTCTGC TAAGTCCTTAATACATCG GTCATTAGATTGCCGTGT GGCGGTTCAGCCCGCCATTGCTATGACGCAAACA GGGGTTAGGTTCTCTCCCCTGGTGGGGATGACAA GAAATGGAAGTTTAGCTC AGTACATCGTCAGCAAACGTAATACCTTGACATA AGTCTTAGTATGGTGCAG GCGGCGCTGCTCATGCAA AGACCGTCTTCAGGGCTA GGCGCTGCTCATGCAAGC ACCGTCTTCAGGGCTACTCACTACTGCTTGG AGCCTAGGGGTTGACCTG ATCCGCCGACGAAGGCCT ATAATCTCTTTCAACCGC TGATGTCCATTACTCCTT ATACTACGCTTGGGAGTT CTATGGTGCATCTCCGCG CTACGCTTGGGAGTTTCG TAGTTTCTCAGAACCCTC TGCCAGATTATATTATCTTGATTCCAGCTAGGA TGGCGGTTCAGCCCGCCA GTGTCCAGGCCTACCTTT ACTTATCACAAGGTAAGGTTTCCGATAGTCGTGA TAATCTCTTTCAACCGCG TCTCGTCGGGGA

In [ ]:
AGA ATG CAT GAT TGGA TGT

In [59]:
def extract_vertices(kmers, k):
    vertex_set = {kmer[:k - 1] for kmer in kmers} | {kmer[1:] for kmer in kmers}
    return {v: i for i, v in enumerate(vertex_set)}


def build_graph(kmers, k, vertices):
    n = len(vertices)
    edges = [[] for _ in range(n)]
    for kmer in kmers:
        fr = vertices[kmer[:k - 1]]
        to = vertices[kmer[1:]]
        edges[fr].append(to)
    return edges


def find_max_nonbranching_paths(edges):
    n = len(edges)
    in_degs = [0 for _ in range(n)]
    for out_edges in edges:
        for v in out_edges:
            in_degs[v] += 1
    out_degs = [len(out_edges) for out_edges in edges]

    nonbranching_paths = []
    for v in range(n):
        if in_degs[v] == 1 and out_degs[v] == 1:
            continue
        if out_degs[v] == 0:
            continue
        for to in edges[v]:
            path = [v, to]
            next = to
            while in_degs[next] == 1 and out_degs[next] == 1:
                next = edges[next][0]
                path.append(next)
                if len(path) == n: # cycle encountered
                    break
            nonbranching_paths.append(path)
    return nonbranching_paths


In [ ]:
patterns = [s.strip() for s in list(open("rosalind_ba3k.txt", "r"))]

In [89]:
def main():
    kmers = [s.strip() for s in list(open("rosalind_ba3k.txt", "r"))]
    k = len(kmers[0])
    vertices = extract_vertices(kmers, k)
    descriptors = {i: v for v, i in vertices.items()}
    edges = build_graph(kmers, k, vertices)
    nonbranching_paths = find_max_nonbranching_paths(edges)
    contigs = []
    for path in nonbranching_paths:
        contig = descriptors[path[0]] + ''.join([descriptors[v][-1] for v in path[1:]])
        contigs.append(contig)
    print(*contigs)

In [90]:
if __name__ == '__main__':
    main()

CCAGCCTCTGCGGTGTCGTGAAGCCGACCCCTAAAAAAAC AGATGGGGTGGAATATTTCCGGT AGATGGGGTGGAATATTTCCGGT GGGCGTCCTTGTGCTACATCAAT GGGCGTCCTTGTGCTACATCAAT CACGCATATAAACTGTATCACTC CACGCATATAAACTGTATCACTC CATTCTTAGTCTCCAATAATTAG CATTCTTAGTCTCCAATAATTAG GAGTAATCGAACTCTTCAACGGT GAGTAATCGAACTCTTCAACGGT TGCAGTTCATTTCAACCGGGGAA TGCAGTTCATTTCAACCGGGGAA GGTTTCCAGTAGCTTTATAACCG GGTTTCCAGTAGCTTTATAACCG GTAATGCATCATGTCCATGGCATCACCTCAGCTGACCCGG GGTACAAGGAGTACCTGCGCCGC GGTACAAGGAGTACCTGCGCCGC AACAGATCCTGTCAAGTCGCTTT AACAGATCCTGTCAAGTCGCTTT AGAGAATCCGAAGAAGTTCTCTA AGAGAATCCGAAGAAGTTCTCTA TAGCGCCCTTCGGCGCAGCTCCA TAGCGCCCTTCGGCGCAGCTCCA TGGTAATGCATCATGTCCATGGC TGGTAATGCATCATGTCCATGGC CTAGACACGTTAACAAAAGAACAGATCCTGTCAAGTCGCT AGGGCCGCCCACAGCTCGCCCAT AGGGCCGCCCACAGCTCGCCCAT GTACCGTAGGAGTAAGCGGGGCCGCATCACGTTGAGAAACTAT CCGGGTCTCTATCACAGGTGCATTTCTTACCGGTCAGCAC AGCCGCTCGAAGCGATGATGAGG AGCCGCTCGAAGCGATGATGAGG TTTACTAGACACGTTAACAAAAG TTTACTAGACACGTTAACAAAAG GTCAGGAGTAGGCCAGTTAGACT GTCAGGAGTAGGCCAGTTAGACT CAGGAGTAGGCCAGTTAGACTTC 

In [81]:
def get_vertex_set(patterns, k):
    # V = (all prefixes) U (all suffixes)
    V = {kmer[:k - 1] for kmer in patterns} | {kmer[1:] for kmer in patterns}
    return {v: i for i, v in enumerate(V)}

In [82]:
def build_graph(patterns, k, vertices):
    n = len(vertices)
    Adj = [[] for _ in range(n)]
    for kmer in patterns:
        prefix = vertices[kmer[:k - 1]]
        suffix = vertices[kmer[1:]]
        Adj[prefix].append(suffix)
    return Adj

In [83]:
# Find contigs by max non-branching path
def find_max_nonbranching_paths(Adj):
    n = len(Adj)
    in_degree = [0 for _ in range(n)]
    for out_edges in Adj:
        for u in out_edges:
            in_degree[u] += 1
    out_degree = [len(out_edges) for out_edges in Adj]

    non_branching_paths = []
    for u in range(n):
        if in_degree[u] == 1 and out_degree[u] == 1:
            continue
        if out_degree[u] == 0:
            continue
        for v in Adj[u]:
            path = [u, v]
            w = v
            while in_degree[w] == 1 and out_degree[w] == 1:
                w = Adj[w][0]
                path.append(w)
                if len(path) == n: # cycle encountered
                    break
            non_branching_paths.append(path)
    return non_branching_paths

In [84]:
def find_contigs(patterns):
    k = len(patterns[0])
    vertices = get_vertex_set(patterns, k)
    id_to_vertex = {i: v for v, i in vertices.items()}
    Adj = build_graph(patterns, k, vertices)
    non_branching_paths = find_max_nonbranching_paths(Adj)

    contigs = []
    for path in non_branching_paths:
        contig = id_to_vertex[path[0]] + ''.join([id_to_vertex[v][-1] for v in path[1:]])
        contigs.append(contig)

    return contigs

In [91]:
def main():
    
    file = open('rosalind_ba3k.txt', 'r')
    
    patterns = []
    for s in file:
        patterns.append(s.strip())
    
    
    print(*find_contigs(patterns))

    file.close()

In [92]:
if __name__ == '__main__':
    main()

CCAGCCTCTGCGGTGTCGTGAAGCCGACCCCTAAAAAAAC AGATGGGGTGGAATATTTCCGGT AGATGGGGTGGAATATTTCCGGT GGGCGTCCTTGTGCTACATCAAT GGGCGTCCTTGTGCTACATCAAT CACGCATATAAACTGTATCACTC CACGCATATAAACTGTATCACTC CATTCTTAGTCTCCAATAATTAG CATTCTTAGTCTCCAATAATTAG GAGTAATCGAACTCTTCAACGGT GAGTAATCGAACTCTTCAACGGT TGCAGTTCATTTCAACCGGGGAA TGCAGTTCATTTCAACCGGGGAA GGTTTCCAGTAGCTTTATAACCG GGTTTCCAGTAGCTTTATAACCG GTAATGCATCATGTCCATGGCATCACCTCAGCTGACCCGG GGTACAAGGAGTACCTGCGCCGC GGTACAAGGAGTACCTGCGCCGC AACAGATCCTGTCAAGTCGCTTT AACAGATCCTGTCAAGTCGCTTT AGAGAATCCGAAGAAGTTCTCTA AGAGAATCCGAAGAAGTTCTCTA TAGCGCCCTTCGGCGCAGCTCCA TAGCGCCCTTCGGCGCAGCTCCA TGGTAATGCATCATGTCCATGGC TGGTAATGCATCATGTCCATGGC CTAGACACGTTAACAAAAGAACAGATCCTGTCAAGTCGCT AGGGCCGCCCACAGCTCGCCCAT AGGGCCGCCCACAGCTCGCCCAT GTACCGTAGGAGTAAGCGGGGCCGCATCACGTTGAGAAACTAT CCGGGTCTCTATCACAGGTGCATTTCTTACCGGTCAGCAC AGCCGCTCGAAGCGATGATGAGG AGCCGCTCGAAGCGATGATGAGG TTTACTAGACACGTTAACAAAAG TTTACTAGACACGTTAACAAAAG GTCAGGAGTAGGCCAGTTAGACT GTCAGGAGTAGGCCAGTTAGACT CAGGAGTAGGCCAGTTAGACTTC 